In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.1 MB/s eta 0:00:00


In [3]:
# imports
import pandas as pd
from copy import deepcopy
import os
from datasets import Dataset, Features, Value, ClassLabel
import re
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from evaluate import evaluator, combine, load
from transformers import pipeline

In [8]:
# rutas
#parent_dir = #os.path.dirname(os.getcwd())
#path_datos = "/content/drive/Shareddrives/Proyecto PNL/Proyecto PLN 20221125/Proyecto PLN 2022-2/sentencias/Clasificación de Parrafos/scripts_python/parrafos/parrafos.xlsx"#os.path.join(parent_dir, "parrafos", "parrafos.xlsx")
#path_datos = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/parrafos_2023.xlsx"#os.path.join(parent_dir, "parrafos", "parrafos.xlsx")
path_datos = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Compilado parrafos_2023.xlsx"#os.path.join(parent_dir, "parrafos", "parrafos.xlsx")
path_resultados = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Resultados"#os.path.join(parent_dir, "resultados")
path_modelo_salida = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/modelo_salida"

path_resultados_QA = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Resultados_QA"#os.path.join(parent_dir, "resultados")
path_modelo_salida_QA = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/modelo_salida_QA"

In [4]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# funciones

def categoria(old_label):
  new_label = None
  if old_label in ["Antecedentes de hecho y hechos probados", "Fundamentos de Derecho"]:
    new_label = 0
  if old_label == "Encabezamiento":
    new_label = 1
  if old_label == "Parte dispositiva y fallo":
    new_label = 2
  return(new_label)


def split_df(df_datos, train = 0.8):
  df = deepcopy(df_datos)
  df_train = df.sample(frac = train)
  df_validation = df.drop(df_train.index)
  df_train.reset_index(drop = True, inplace = True)
  df_validation.reset_index(drop = True, inplace = True)
  return(df_train, df_validation)


def recortar(parrafo, max_len):
    split_parrafo = parrafo.split(" ")
    if len(split_parrafo) > max_len:
        new_parrafo = " ".join(split_parrafo[:max_len])
        return new_parrafo
    else:
        return parrafo

def tokenize_function(parrafos):
    #return tokenizer(parrafos["text"], padding = "max_length", truncation = True)
    return tokenizer(parrafos["text"], padding = True, truncation = True, max_length = 512, add_special_tokens = True)
    #return tokenizer(parrafos["text"], truncation = True)

#P1

In [10]:
# carga de datos
datos = pd.read_excel(path_datos, sheet_name = "Datos", dtype = {"Radicado": "string"})
datos["Parrafo"] = datos["Parrafo"].astype(str)  # Convertir los valores a tipo cadena
datos["Parrafo"] = datos["Parrafo"].apply(lambda x: re.sub("\\s{2,}", " ", x)).str.strip()
datos["Parrafo"] = datos["Parrafo"].apply(lambda x: recortar(x, 511))
datos.dtypes

Id_parrafo     int64
Parrafo       object
P1            object
R1_ini         int64
R1_fin         int64
P2            object
R2_ini         int64
R2_fin         int64
dtype: object

In [12]:
datos = datos[datos['R1_ini'].notnull()]
datos.head()

,Id_parrafo,Parrafo,P1,R1_ini,R1_fin,P2,R2_ini,R2_fin
0,1,JUZGADO PRIMERO CIVIL DEL CIRCUITO ESPECIALIZA...,Cual en el radicado,0,0,Cual es el sentido de la decision,0,0
1,2,"Medellín, treinta de marzo de dos mil veintidós",Cual en el radicado,0,0,Cual es el sentido de la decision,0,0
2,3,Proceso: Restitución y Formalización de tierras,Cual en el radicado,0,0,Cual es el sentido de la decision,0,0
3,4,Solicitante: Francisco Javier Giraldo Giraldo,Cual en el radicado,0,0,Cual es el sentido de la decision,0,0
4,5,Radicado: 050003121001202000039 00,Cual en el radicado,10,33,Cual es el sentido de la decision,0,0


In [13]:
datos['P1'] = datos['P1'].astype(str).str.encode('utf-8')

In [14]:
datos['P1'] = datos['P1'].str.decode('utf-8')

In [15]:
import pandas as pd
from datasets import Dataset

DatosSet = Dataset.from_pandas(datos)

In [16]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
#LOgearse en Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [18]:
DatosSet = DatosSet.train_test_split(test_size=0.2)

In [19]:
DatosSet['train']

Dataset({
    features: ['Id_parrafo', 'Parrafo', 'P1', 'R1_ini', 'R1_fin', 'P2', 'R2_ini', 'R2_fin', '__index_level_0__'],
    num_rows: 2088
})

In [20]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from transformers import AutoTokenizer
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

# Cargar el modelo preentrenado y el tokenizador en español
model_name = 'distilbert-base-multilingual-cased'
#model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['distilbert.transformer.layer.1.sa_layer_norm.bias', 'distilbert.transformer.layer.3.attention.k_lin.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.transformer.layer.3.attention.out_lin.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.3.ffn.lin2.weight

In [24]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["P1"]]
    encoded_inputs = tokenizer(
        questions,
        examples["Parrafo"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt"
    )

    input_ids = encoded_inputs.input_ids
    attention_mask = encoded_inputs.attention_mask

    start_positions = []
    end_positions = []

    for i in range(len(input_ids)):
        # Obtener los índices de inicio y fin de la respuesta dentro de los tokens
        start_char = examples["R1_ini"][i]
        end_char = examples["R1_fin"][i]
        offsets = tokenizer.decode(input_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=False)

        start_token_idx = None
        end_token_idx = None

        current_offset = 0
        for j, token in enumerate(offsets):
            if token != " ":
                if start_char is not None and start_char >= current_offset and start_char < current_offset + len(token):
                    start_token_idx = j
                if end_char is not None and end_char >= current_offset and end_char < current_offset + len(token):
                    end_token_idx = j
                current_offset += len(token)

        # Guardar las posiciones de inicio y fin
        if start_token_idx is not None and end_token_idx is not None:
            start_positions.append(start_token_idx)
            end_positions.append(end_token_idx)
        else:
            start_positions.append(-1)
            end_positions.append(-1)

    encoded_inputs["start_positions"] = start_positions
    encoded_inputs["end_positions"] = end_positions

    return encoded_inputs

In [25]:
tokenized_train_data = DatosSet.map(preprocess_function, batched=True)

Map:   0%|          | 0/2088 [00:00<?, ? examples/s]

Map:   0%|          | 0/523 [00:00<?, ? examples/s]

In [26]:
#TensorFlow
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [27]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_train_data["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [30]:
from transformers import TFAutoModelForQuestionAnswering

#model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [31]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_data["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [32]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_train_data["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [33]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [36]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    #output_dir=path_resultados_QA,
    output_dir="CristianDiaz/ModeloSentenciasQA_1Preg",
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/CristianDiaz/ModeloSentenciasQA_1Preg into local empty directory.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])

Epoch 1/3
 11/130 [=>............................] - ETA: 1:35:11 - loss: 0.2467

# Inferencia

In [ ]:
Pregunta = "cual es el radicado"
Parrafo = "SENTENCIA No: 2-R RADICADO: 05000312100120190002401 PROCESO: ESPECIAL DE RESTITUCIÓN DE TIERRAS SOLICITANTE: MARÍA LETICIA RAMÍREZ RAMÍREZ OPOSITOR: GLORIA HELENA GUTIÉRREZ BILBAO SINOPSIS: Se encontraron probados los presupuestos axiológicos para amparar el derecho fundamental a la restitución, por la vía de la compensación. No prospera la oposición ni se reúnen los requisitos para reconocer la segunda ocupación. Se reitera precedente en torno al carácter especial y breve del proceso de restitución, y el tratamiento de los bienes baldíos en restitución de tierras."

In [ ]:
# Pipeline
from transformers import pipeline

question_answerer = pipeline("question-answering", model="CristianDiaz/ModeloSentenciasQA_1Preg")
question_answerer(question=Pregunta, context=Parrafo)


In [ ]:
## TensorFlow ##
#Tokeniza y deviuelve tensores
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("CristianDiaz/ModeloSentenciasQA_1Preg")
inputs = tokenizer(Pregunta, Parrafo, return_tensors="tf")

In [ ]:
#Inputs al modelo y devuelve los registros:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("CristianDiaz/ModeloSentenciasQA_1Preg")
outputs = model(**inputs)

In [ ]:
#mayor probabilidad de la salida del modelo para las posiciones de inicio y finalización
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

In [ ]:
#Decodifique los tokens predichos para obtener la respuesta
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

#2 Preguntas

In [31]:
path_datos2 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Copia de parrafos_2023_SinP2.xlsx"#os.path.join(parent_dir, "parrafos", "parrafos.xlsx")
path_resultados_QA3 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Resultados_QA3"#os.path.join(parent_dir, "resultados")
path_modelo_salida_QA3 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/modelo_salida_QA3"
path_modelo_salida_QA4 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/modelo_salida_QA4"


In [32]:
# carga de datos
datos2 = pd.read_excel(path_datos2, sheet_name = "Datos", dtype = {"Radicado": "string", "Categoria": "category"})
datos2["Parrafo"] = datos2["Parrafo"].apply(lambda x: re.sub("\\s{2,}", " ", x)).str.strip()
datos2["Parrafo"] = datos2["Parrafo"].apply(lambda x: recortar(x, 511))
datos2.dtypes

Radicado          string
ID Categoria       int64
No. Parrafo        int64
Parrafo           object
Categoria       category
caracteres         int64
P1                object
R1_ini           float64
R1_fin           float64
P2                object
R2_ini           float64
R2_fin           float64
dtype: object

In [33]:
datos2 = datos2[datos2['R1_ini'].notna() | datos2['R2_ini'].notna()]
columnas = ["R1_ini", "R1_fin", "R2_ini", "R2_fin"]
datos2[columnas] = datos2[columnas].fillna(0)
datos2.head()

<ipython-input-33-ca0689493e2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos2[columnas] = datos2[columnas].fillna(0)


,Radicado,ID Categoria,No. Parrafo,Parrafo,Categoria,caracteres,P1,R1_ini,R1_fin,P2,R2_ini,R2_fin
8,86001312100120200017500,0,9,Radicado: 860013121001-2020-00175-00,Encabezamiento,35,Cual en el radicado,9.0,35.0,Cual es el sentido de la decision,0.0,0.0
121,86001312100120200017500,3,5,SEGUNDO: ORDENAR al Grupo Cumplimiento de Órde...,Parte dispositiva y fallo,370,Cual en el radicado,0.0,0.0,Cual es el sentido de la decision,170.0,197.0
159,68081312100120190001601,0,2,Radicado: 68081312100120190001601.\nSentencia:...,Encabezamiento,55,Cual en el radicado,10.0,33.0,Cual es el sentido de la decision,0.0,0.0
308,68081312100120190001601,3,6,TERCERO. RECONOCER a favor de María Rojas y la...,Parte dispositiva y fallo,829,Cual en el radicado,0.0,0.0,Cual es el sentido de la decision,151.0,179.0
345,76001312100320200008000,0,5,Radicado: 760013121003 2020 00080 00 - Sentenc...,Encabezamiento,57,Cual en el radicado,10.0,35.0,Cual es el sentido de la decision,0.0,0.0


In [34]:
datos2['P1'] = datos2['P1'].astype(str).str.encode('utf-8')
datos2['P1'] = datos2['P1'].str.decode('utf-8')
datos2['Categoria'] = datos2['Categoria'].astype(str)

In [35]:
import pandas as pd
from datasets import Dataset

DatosSet2 = Dataset.from_pandas(datos2)

In [36]:
DatosSet2 = DatosSet2.train_test_split(test_size=0.2)

In [37]:
DatosSet2['train'][0]

{'Radicado': '86001312100120200008700',
 'ID Categoria': 0,
 'No. Parrafo': 4,
 'Parrafo': 'Referencia: Solicitud Restitución y Formalización de Tierras\nSolicitante: Erly Cruz Verano\nVinculados: Agencia Nacional de Hidrocarburos - Departamento del\nPutumayo - Municipio de Valle del Guamuez - Personas\nIndeterminadas Radicado: 860013121001-2020-00087-00',
 'Categoria': 'Encabezamiento',
 'caracteres': 260,
 'P1': '246',
 'R1_ini': 260.0,
 'R1_fin': 0.0,
 'P2': 'Cual es el sentido de la decision',
 'R2_ini': 0.0,
 'R2_fin': 0.0,
 '__index_level_0__': 1264}

####Preprosesamiento

In [75]:
from transformers import AutoTokenizer
model_name ='dccuchile/bert-base-spanish-wwm-uncased'
#model_name =  'bert-large-uncased-whole-word-masking-finetuned-squad'
#model_name =  'mrm8488/bert-small-finetuned-squadv2-es'
#model_name = 'mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es' # pytorch
#model_name = 'distilbert-base-multilingual-cased'
#model_name = 'mrm8488/bert-small-finetuned-squadv2-es'
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [76]:
def preprocess_function(examples):
    encoded_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "start_positions": [],
        "end_positions": []
    }

    for i in range(1, 2):
        questions_key = f"P{i}"
        start_char_key = f"R{i}_ini"
        end_char_key = f"R{i}_fin"

        if questions_key not in examples or start_char_key not in examples or end_char_key not in examples:
            continue

        questions = [q.strip() for q in examples[questions_key]]
        start_char = examples[start_char_key]
        end_char = examples[end_char_key]

        encoded_inputs_i = tokenizer(
        questions,
        examples["Parrafo"],
        max_length=27,  # Ajusta el valor según tus necesidades
        truncation=True,
        padding="max_length",
        return_tensors="pt"
        )

        input_ids = encoded_inputs_i.input_ids
        attention_mask = encoded_inputs_i.attention_mask

        current_start_positions = []
        current_end_positions = []

        for j in range(len(input_ids)):
            # Obtener los índices de inicio y fin de la respuesta dentro de los tokens
            start_token_idx = None
            end_token_idx = None

            offsets = tokenizer.decode(input_ids[j], skip_special_tokens=True, clean_up_tokenization_spaces=False)
            current_offset = 0

            for k, token in enumerate(offsets):
                if token != " ":
                    if start_char[j] is not None and start_char[j] >= current_offset and start_char[j] < current_offset + len(token):
                        start_token_idx = k
                    if end_char[j] is not None and end_char[j] >= current_offset and end_char[j] < current_offset + len(token):
                        end_token_idx = k
                    current_offset += len(token)

            # Guardar las posiciones de inicio y fin
            if start_token_idx is not None and end_token_idx is not None:
                current_start_positions.append(start_token_idx)
                current_end_positions.append(end_token_idx)
            else:
                current_start_positions.append(-1)
                current_end_positions.append(-1)

        # Extender los valores en el diccionario encoded_inputs
        encoded_inputs["input_ids"].extend(input_ids)
        encoded_inputs["attention_mask"].extend(attention_mask)
        encoded_inputs["start_positions"].extend(current_start_positions)
        encoded_inputs["end_positions"].extend(current_end_positions)

    return encoded_inputs

In [77]:
#Tokenizacion
tokenized_train_data2 = DatosSet2.map(preprocess_function, batched=True)

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

### Fine Tune

In [78]:
#TensorFlow
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [79]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_train_data2["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [80]:
from transformers import TFAutoModelForQuestionAnswering

#model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
#model_name = 'mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
#model_name =  'bert-large-uncased-whole-word-masking-finetuned-squad'
#model_name = 'mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_data2["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [82]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_train_data2["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [83]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [68]:
############### NO CORRER
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=optimizer, loss=loss)


In [84]:
##guardar en Hugging Face
from transformers.keras_callbacks import PushToHubCallback

callback2 = PushToHubCallback(
    output_dir="CristianDiaz/Modelo_Sentencias_URT_3P",
    tokenizer=tokenizer,
)

/content/CristianDiaz/Modelo_Sentencias_URT_3P is already a clone of https://huggingface.co/CristianDiaz/Modelo_Sentencias_URT_3P. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback2])

In [ ]:
########## Pruebas para identificar el error

In [104]:
tokenized_train_data2["test"][2]

{'Radicado': '68081312100120190001601',
 'ID Categoria': 0,
 'No. Parrafo': 2,
 'Parrafo': 'Radicado: 68081312100120190001601.\nSentencia: 3 de 2022.',
 'Categoria': 'Encabezamiento',
 'caracteres': 55,
 'P1': 'Cual en el radicado',
 'R1_ini': 10.0,
 'R1_fin': 33.0,
 'P2': 'Cual es el sentido de la decision',
 'R2_ini': 0.0,
 'R2_fin': 0.0,
 '__index_level_0__': 159,
 'input_ids': [4,
  1596,
  1035,
  1039,
  24435,
  1050,
  5,
  24435,
  1050,
  995,
  1001,
  996,
  30987,
  996,
  9909,
  8361,
  26192,
  8361,
  10132,
  9054,
  26772,
  1000,
  10132,
  1008,
  7064,
  995,
  5],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'start_positions': 13,
 'end_positions': 39}

In [87]:
#etiquetas de -1
import numpy as np

# Supongamos que tienes tus conjuntos de datos tf_train_set y tf_validation_set

# Obtener las etiquetas de los conjuntos de datos
y_train = [item[1] for item in tf_train_set]
y_validation = [item[1] for item in tf_validation_set]

# Convertir las etiquetas a arreglos de numpy para facilitar el procesamiento
y_train_array = np.array(y_train)
y_validation_array = np.array(y_validation)

# Encontrar los índices donde se encuentra la etiqueta -1 en tf_train_set
train_indices = np.where(y_train_array == -1)[0]

# Encontrar los índices donde se encuentra la etiqueta -1 en tf_validation_set
validation_indices = np.where(y_validation_array == -1)[0]

# Verificar si existen etiquetas con valor -1 en tf_train_set
if len(train_indices) > 0:
    print("Se encontraron etiquetas con valor -1 en tf_train_set en los siguientes índices:")
    print(train_indices)
else:
    print("No se encontraron etiquetas con valor -1 en tf_train_set.")

# Verificar si existen etiquetas con valor -1 en tf_validation_set
if len(validation_indices) > 0:
    print("Se encontraron etiquetas con valor -1 en tf_validation_set en los siguientes índices:")
    print(validation_indices)
else:
    print("No se encontraron etiquetas con valor -1 en tf_validation_set.")

In [93]:
import numpy as np

# Obtener algunas muestras de los conjuntos de datos
sample_train_data = tf_train_set.take(10)  # Obtener las primeras 10 muestras de tf_train_set
sample_validation_data = tf_validation_set.take(10)  # Obtener las primeras 10 muestras de tf_validation_set

# Recorrer las muestras y verificar las etiquetas
for data in sample_train_data:
    input_ids, labels = data  # Obtener los datos de entrada y las etiquetas
    label_key = next(iter(labels.keys()))  # Obtener el nombre de la clave de las etiquetas
    label_values = labels[label_key].numpy()  # Obtener los valores de las etiquetas como un arreglo numpy
    if -1 in label_values:
        print("Se encontró una etiqueta con valor -1 en tf_train_set.")
        break

for data in sample_validation_data:
    input_ids, labels = data  # Obtener los datos de entrada y las etiquetas
    label_key = next(iter(labels.keys()))  # Obtener el nombre de la clave de las etiquetas
    label_values = labels[label_key].numpy()  # Obtener los valores de las etiquetas como un arreglo numpy
    if -1 in label_values:
        print("Se encontró una etiqueta con valor -1 en tf_validation_set.")
        break

In [96]:
for data in tf_train_set:
    print(data)
    break

for data in tf_validation_set:
    print(data)
    break

In [97]:
print(len(tf_train_set))
print(len(tf_validation_set))

In [99]:
train_iterator = tf_train_set.as_numpy_iterator()

for data in train_iterator:
    input_ids, labels = data  # Obtener los datos de entrada y las etiquetas
    # Realizar las operaciones necesarias con los datos

In [ ]:
model.save_pretrained(path_modelo_salida_QA4)

##Evaluate
La evaluación para la QA requiere una cantidad significativa de posprocesamiento. Para evitar tomar demasiado tiempo, esta guía omite el paso de evaluación. El entrenador aún calcula la pérdida de evaluación durante el entrenamiento, por lo que no está completamente en la oscuridad sobre el rendimiento de su modelo.

Si tiene más tiempo y está interesado en cómo evaluar su modelo para la respuesta a las preguntas, eche un vistazo al capítulo de respuesta de preguntas del curso 🤗 abrazando!

# Inferencia

##### TensorFlow

In [ ]:
from transformers import TFBertForQuestionAnswering

model = TFBertForQuestionAnswering.from_pretrained(path_modelo_salida_QA4)

#### Pipeline

In [ ]:
question = "cual es el radicado"
context = "SENTENCIA No: 2-R RADICADO: 05000312100120190002401 PROCESO: ESPECIAL DE RESTITUCIÓN DE TIERRAS SOLICITANTE: MARÍA LETICIA RAMÍREZ RAMÍREZ OPOSITOR: GLORIA HELENA GUTIÉRREZ BILBAO SINOPSIS: Se encontraron probados los presupuestos axiológicos para amparar el derecho fundamental a la restitución, por la vía de la compensación. No prospera la oposición ni se reúnen los requisitos para reconocer la segunda ocupación. Se reitera precedente en torno al carácter especial y breve del proceso de restitución, y el tratamiento de los bienes baldíos en restitución de tierras."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(path_modelo_salida_QA4)

In [ ]:
modelo_salida_QA4

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model=model)
question_answerer(question=question, context=context)


In [ ]:
import os
os.listdir(path_modelo_salida_QA4)

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(path_modelo_salida_QA3)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(path_modelo_salida_QA3)
inputs = tokenizer(question, context, return_tensors="tf")

#3 preguntas


In [ ]:
path_datos2 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Copia de parrafos_2023_SinP2.xlsx"#os.path.join(parent_dir, "parrafos", "parrafos.xlsx")
path_resultados_QA3 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/Resultados_QA3"#os.path.join(parent_dir, "resultados")
path_modelo_salida_QA3 = "/content/drive/Shareddrives/Inteligencia Artificial 2023-1/Proyecto Final/2023 Answering Question/modelo_salida_QA3"

In [ ]:
# carga de datos
datos2 = pd.read_excel(path_datos2, sheet_name = "Datos", dtype = {"Radicado": "string", "Categoria": "category"})
datos2["Parrafo"] = datos2["Parrafo"].apply(lambda x: re.sub("\\s{2,}", " ", x)).str.strip()
datos2["Parrafo"] = datos2["Parrafo"].apply(lambda x: recortar(x, 511))
datos2.dtypes

In [ ]:
datos2 = datos2[datos2['R1_ini'].notna() | datos2['R2_ini'].notna()]
columnas = ["R1_ini", "R1_fin", "R2_ini", "R2_fin"]
datos2[columnas] = datos2[columnas].fillna(0)
datos2.head()

In [ ]:
datos2['P1'] = datos2['P1'].astype(str).str.encode('utf-8')
datos2['P1'] = datos2['P1'].str.decode('utf-8')
datos2['Categoria'] = datos2['Categoria'].astype(str)

In [ ]:
import pandas as pd
from datasets import Dataset

DatosSet2 = Dataset.from_pandas(datos2)

In [ ]:
DatosSet2 = DatosSet2.train_test_split(test_size=0.2)

In [ ]:
DatosSet2['train'][0]

####Preprosesamiento

In [ ]:
def preprocess_function(examples):
    encoded_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "start_positions": [],
        "end_positions": []
    }

    for i in range(1, 2):
        questions_key = f"P{i}"
        start_char_key = f"R{i}_ini"
        end_char_key = f"R{i}_fin"

        if questions_key not in examples or start_char_key not in examples or end_char_key not in examples:
            continue

        questions = [q.strip() for q in examples[questions_key]]
        start_char = examples[start_char_key]
        end_char = examples[end_char_key]

        encoded_inputs_i = tokenizer(
        questions,
        examples["Parrafo"],
        max_length=27,  # Ajusta el valor según tus necesidades
        truncation=True,
        padding="max_length",
        return_tensors="pt"
        )

        input_ids = encoded_inputs_i.input_ids
        attention_mask = encoded_inputs_i.attention_mask

        current_start_positions = []
        current_end_positions = []

        for j in range(len(input_ids)):
            # Obtener los índices de inicio y fin de la respuesta dentro de los tokens
            start_token_idx = None
            end_token_idx = None

            offsets = tokenizer.decode(input_ids[j], skip_special_tokens=True, clean_up_tokenization_spaces=False)
            current_offset = 0

            for k, token in enumerate(offsets):
                if token != " ":
                    if start_char[j] is not None and start_char[j] >= current_offset and start_char[j] < current_offset + len(token):
                        start_token_idx = k
                    if end_char[j] is not None and end_char[j] >= current_offset and end_char[j] < current_offset + len(token):
                        end_token_idx = k
                    current_offset += len(token)

            # Guardar las posiciones de inicio y fin
            if start_token_idx is not None and end_token_idx is not None:
                current_start_positions.append(start_token_idx)
                current_end_positions.append(end_token_idx)
            else:
                current_start_positions.append(-1)
                current_end_positions.append(-1)

        # Extender los valores en el diccionario encoded_inputs
        encoded_inputs["input_ids"].extend(input_ids)
        encoded_inputs["attention_mask"].extend(attention_mask)
        encoded_inputs["start_positions"].extend(current_start_positions)
        encoded_inputs["end_positions"].extend(current_end_positions)

    return encoded_inputs

In [ ]:
from transformers import AutoTokenizer

model_name = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
#Tokenizacion
tokenized_train_data2 = DatosSet2.map(preprocess_function, batched=True)

In [ ]:
#TensorFlow
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_train_data2["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_data2["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_train_data2["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
##guardar en Hugging Face
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=path_resultados_QA3,
    tokenizer=tokenizer,
)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])